In [1]:
import numpy as np
import itertools
#
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
# from WolframClientForPython.wolframclient.evaluation import WolframAlphaSession
import heapq
import pandas as pd
import math

In [2]:
arr = np.array([[(80, 6), (50, 4), (83, 7), (31, 2), (60, 4)],
                [(89, 8), (10, 1), (37, 3), (70, 4), (90, 10)],
                [(17, 1), (40, 3), (73, 4), (100, 15), (20, 2)],
                [(41, 3), (79, 5), (23, 2), (47, 3), (30, 2)]])

print(f"arr shape: {arr.shape}")

arr shape: (4, 5, 2)


In [3]:
thresh = np.zeros(arr.shape[:2])
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        mult, inhabitants = arr[i, j]
        thresh[i, j] = (mult - 5 * inhabitants) / 500
with np.printoptions(precision=3, suppress=True):
    print(thresh)

[[0.1   0.06  0.096 0.042 0.08 ]
 [0.098 0.01  0.044 0.1   0.08 ]
 [0.024 0.05  0.106 0.05  0.02 ]
 [0.052 0.108 0.026 0.064 0.04 ]]


In [4]:
thresh = np.zeros(arr.shape[:2])
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        mult, inhabitants = arr[i, j]
        thresh[i, j] = (mult - 10 * inhabitants) / 1000
with np.printoptions(precision=3, suppress=True):
    print(thresh)

[[ 0.02   0.01   0.013  0.011  0.02 ]
 [ 0.009  0.     0.007  0.03  -0.01 ]
 [ 0.007  0.01   0.033 -0.05   0.   ]
 [ 0.011  0.029  0.003  0.017  0.01 ]]


In [ ]:
def maximin1():
    """Solves the maximin optimization problem for a single expedition.

    Parameters
    ----------
    
    Returns
    -------
    argmax : list of tuple
        Maximizers.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    argmax = []
    for mult, inhabitants in arr.reshape(-1, 2):
        val = mult / (inhabitants + 100) * 10000
        if math.isclose(val, max_val):
            argmax.append((mult, inhabitants))
        elif val > max_val:
            argmax = [(mult, inhabitants)]
            max_val = val
    return (argmax, max_val)

In [6]:
maximin1()

([(np.int64(100), np.int64(15))], np.float64(8695.652173913044))

In [7]:
session = WolframLanguageSession(kernel="/Applications/Wolfram Engine.app/Contents/MacOS/WolframKernel")

def maximize2():
    """Solves the maximin optimization problem for two expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    for (mult1, hunt1), (mult2, hunt2) in itertools.combinations(arr.reshape((-1, 2)), 2):
        val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2), 0 <= p1, 0 <= p2, p1 + p2 <= 1}}, {{p1, p2}}]'))[0]
        val = -50000 + val_temp * 10000
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), max_val)

In [9]:
maximize2()

((np.int64(89), np.int64(8)), (np.int64(100), np.int64(15)), -19294.3398722117)

In [12]:
def maximize3():
    """Solves the maximin optimization problem for three expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    argmax3 : tuple
        Third expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    max_mult3, max_hunt3 = None, None
    for (mult1, hunt1), (mult2, hunt2), (mult3, hunt3) in itertools.combinations(arr.reshape((-1, 2)), 3):
        val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2) + {mult3}/({hunt3} + 100*p3), 0<=p1, 0<=p2, 0<=p3, p1+p2+p3 <= 1}}, {{p1,p2,p3}}]'))[0]
        val = -100000 + val_temp * 10000
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_mult3, max_hunt3 = mult3, hunt3
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), (max_mult3, max_hunt3), max_val)

In [13]:
maximize3()

((np.int64(89), np.int64(8)),
 (np.int64(90), np.int64(10)),
 (np.int64(100), np.int64(15)),
 -37111.68448500797)

In [14]:
ratios = np.zeros(arr.shape[:2])
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        mult, hunt = arr[i, j]
        ratios[i, j] = mult/hunt
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

Ratios:
[[13.33 12.5  11.86 15.5  15.  ]
 [11.12 10.   12.33 17.5   9.  ]
 [17.   13.33 18.25  6.67 10.  ]
 [13.67 15.8  11.5  15.67 15.  ]]


In [15]:
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior:", shares, sep='\n')

Natural prior:
[[0.05  0.047 0.045 0.058 0.057]
 [0.042 0.038 0.047 0.066 0.034]
 [0.064 0.05  0.069 0.025 0.038]
 [0.052 0.06  0.043 0.059 0.057]]


In [16]:
ratios = np.zeros(arr.shape[:2])
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        mult, hunt = arr[i, j]
        ratios[i, j] = mult/hunt
ratios = ratios**5
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior with exponent 5:", shares, sep='\n')

Natural prior with exponent 5:
[[0.034 0.025 0.019 0.073 0.062]
 [0.014 0.008 0.023 0.134 0.005]
 [0.116 0.034 0.166 0.001 0.008]
 [0.039 0.081 0.016 0.077 0.062]]


In [19]:
def fee(n):
    """Compute the fee for a total of n expeditions.

    Parameters
    ----------
    n : int
        Number of expeditions.
    
    Returns
    -------
    float
        Fee.
    """
    if n == 1:
        return 0
    if n == 2:
        return -50000
    if n == 3:
        return -150000

def payoff(mults, hunts, shares):
    """Compute the final profit after the expeditions.

    Parameters
    ----------
    mults : list of int
        Multipliers for each destination.
    hunts : list of int
        Hunters for each destination.
    shares : list of int
        Shares for each destination.
    
    Returns
    -------
    float
        Profit.
    """
    return 10000 * sum([mult/(hunt + 100*share) for (mult, hunt, share) in zip(mults, hunts, shares)]) + fee(len(mults))

def maximize_prior_top(shares, k):
    """Given the prior, compute solutions that yield top k profits.

    Parameters
    ----------
    shares : list of int
        Shares for each destination.
    k : int
        Number of solutions
    
    Returns
    -------
    list of tuple
        Top k profits and optimal expeditions.
    """
    datas = [(mult, hunt, share) for ((mult, hunt), share) in zip(arr.reshape((-1, 2)), shares.reshape(-1))]
    print(datas)
    heap = []
    iterables = [itertools.combinations(datas, n_exp) for n_exp in range(1, 4)]
    for (i, data) in enumerate(itertools.chain.from_iterable(iterables)):
        mults = [tupl[0] for tupl in data]
        hunts = [tupl[1] for tupl in data]
        shares = [tupl[-1] for tupl in data]
        val = payoff(mults, hunts, shares)
        expeditions = list(zip(mults, hunts))
        if i < k:
            heapq.heappush(heap, (val, expeditions))
        elif val > heap[0][0]:
            heapq.heappop(heap)
            heapq.heappush(heap, (val, expeditions))
    return sorted(heap, reverse=True)

In [20]:
for i in range(10):
    shares = ratios**i
    shares = shares / np.sum(shares)
    res = maximize_prior_top(shares, 1)
    
    # convert res values to int
    res = [(int(x[0]), [(int(y[0]), int(y[1])) for y in x[1]]) for x in res]
    print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1])

[(np.int64(80), np.int64(6), np.float64(0.05)), (np.int64(50), np.int64(4), np.float64(0.05)), (np.int64(83), np.int64(7), np.float64(0.05)), (np.int64(31), np.int64(2), np.float64(0.05)), (np.int64(60), np.int64(4), np.float64(0.05)), (np.int64(89), np.int64(8), np.float64(0.05)), (np.int64(10), np.int64(1), np.float64(0.05)), (np.int64(37), np.int64(3), np.float64(0.05)), (np.int64(70), np.int64(4), np.float64(0.05)), (np.int64(90), np.int64(10), np.float64(0.05)), (np.int64(17), np.int64(1), np.float64(0.05)), (np.int64(40), np.int64(3), np.float64(0.05)), (np.int64(73), np.int64(4), np.float64(0.05)), (np.int64(100), np.int64(15), np.float64(0.05)), (np.int64(20), np.int64(2), np.float64(0.05)), (np.int64(41), np.int64(3), np.float64(0.05)), (np.int64(79), np.int64(5), np.float64(0.05)), (np.int64(23), np.int64(2), np.float64(0.05)), (np.int64(47), np.int64(3), np.float64(0.05)), (np.int64(30), np.int64(2), np.float64(0.05))]
Exponent: 0 Profit: 110111.00 Optimal expeditions: [(73,